## Part 3

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
from transformers import BertModel, BertTokenizer
import json
import pandas as pd
import gzip
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
import pandas as pd
import json

# Chemin vers le fichier JSON
chemin_fichier = '/kaggle/input/amazon-fashion-51/AMAZON_FASHION_5.json'

# Liste pour stocker les données JSON individuelles
donnees_json = []

# Lecture du fichier JSON ligne par ligne
with open(chemin_fichier, 'r') as fichier:
    for ligne in fichier:
        donnees_json.append(json.loads(ligne))

# Création d'un DataFrame à partir des données JSON
df = pd.DataFrame(donnees_json)

# Filtrer les lignes où "reviewText" est une chaîne de caractères
df = df[df["reviewText"].apply(lambda x: isinstance(x, str))]

# Afficher les premières lignes du DataFrame
print(df.head())


   overall  verified  reviewTime     reviewerID        asin  \
0      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
1      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
2      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
3      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
4      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   

                                               style reviewerName  \
0   {'Size:': ' Big Boys', 'Color:': ' Blue/Orange'}     Tonya B.   
1  {'Size:': ' Big Boys', 'Color:': ' Black (3746...     Tonya B.   
2  {'Size:': ' Big Boys', 'Color:': ' Blue/Gray L...     Tonya B.   
3  {'Size:': ' Big Boys', 'Color:': ' Blue (37867...     Tonya B.   
4     {'Size:': ' Big Boys', 'Color:': ' Blue/Pink'}     Tonya B.   

                 reviewText     summary  unixReviewTime vote image  
0  Great product and price!  Five Stars      1441324800  NaN   NaN  
1  Great product and price!  Five Stars      1441324800  NaN   NaN  

In [6]:
input_texts = df["reviewText"].tolist()
labels = df["overall"].tolist()


In [7]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
encoded_inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors='pt')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)
dataset = TensorDataset(encoded_inputs['input_ids'], encoded_inputs['attention_mask'], labels)

train_size = int(0.8 * len(dataset))
eval_size = len(dataset) - train_size
train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])


/tmp/ipykernel_34/3652536891.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)


In [11]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device)
optimizer = torch.optim.AdamW(bert_model.parameters(), lr=2e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
num_epochs = 3
for epoch in range(num_epochs):
    # Training
    bert_model.train()
    total_train_loss = 0.0
    num_train_batches = 0
    for batch in train_loader:
        input_ids, attention_mask, label = [t.to(device) for t in batch]
        optimizer.zero_grad()
        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        num_train_batches += 1
    avg_train_loss = total_train_loss / num_train_batches

    # Evaluation
    bert_model.eval()
    total_eval_loss = 0.0
    num_eval_batches = 0
    with torch.no_grad():
        for batch in eval_loader:
            input_ids, attention_mask, label = [t.to(device) for t in batch]
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask, labels=label)
            total_eval_loss += outputs.loss.item()
            num_eval_batches += 1
    avg_eval_loss = total_eval_loss / num_eval_batches

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Evaluation Loss: {avg_eval_loss:.4f}")

Epoch [1/3], Training Loss: 1.1442, Evaluation Loss: 0.1197
Epoch [2/3], Training Loss: 0.0801, Evaluation Loss: 0.0524
Epoch [3/3], Training Loss: 0.0507, Evaluation Loss: 0.0459


In [14]:
review_text = "I dont like it"

tokenized_review = tokenizer(review_text, padding=True, truncation=True, return_tensors='pt')

input_ids = tokenized_review['input_ids'].to(device)
attention_mask = tokenized_review['attention_mask'].to(device)

with torch.no_grad():
    bert_model.eval()
    outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
    predicted_rating = outputs.logits.item()

print(f"Predicted rating: {predicted_rating:.2f}")

Predicted rating: 2.91


#### A rating of 2.91 could suggest a slightly negative sentiment or a neutral sentiment leaning towards negativity, depending on the scale or context of the rating system. It's important to note that the interpretation of the predicted rating may vary based on the specific dataset, task, and criteria used for sentiment analysis or regression.

In [15]:
review_text = "This is nice"

tokenized_review = tokenizer(review_text, padding=True, truncation=True, return_tensors='pt')

input_ids = tokenized_review['input_ids'].to(device)
attention_mask = tokenized_review['attention_mask'].to(device)

with torch.no_grad():
    bert_model.eval()
    outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
    predicted_rating = outputs.logits.item()

print(f"Predicted rating: {predicted_rating:.2f}")

Predicted rating: 4.47


#### AA rating of 4.47 on an assumed scale suggests a highly positive sentiment, indicating that the input text is perceived as favorable or enjoyable by the model. However, it's important to consider the specific rating scale and context of the sentiment analysis task to interpret the predicted rating accurately.
